In [322]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_rows', 540)

### CSV file

In [314]:
df1 = pd.read_csv('analytic_data2019.csv', skiprows=[1,2])

In [315]:
df1.shape

(3193, 534)

In [316]:
list(df1.columns)

['State FIPS Code',
 'County FIPS Code',
 '5-digit FIPS Code',
 'State Abbreviation',
 'Name',
 'Release Year',
 'County Ranked (Yes=1/No=0)',
 'Premature death raw value',
 'Premature death numerator',
 'Premature death denominator',
 'Premature death CI low',
 'Premature death CI high',
 'Premature death (Black)',
 'Premature death (Hispanic)',
 'Premature death (White)',
 'Poor or fair health raw value',
 'Poor or fair health numerator',
 'Poor or fair health denominator',
 'Poor or fair health CI low',
 'Poor or fair health CI high',
 'Poor physical health days raw value',
 'Poor physical health days numerator',
 'Poor physical health days denominator',
 'Poor physical health days CI low',
 'Poor physical health days CI high',
 'Poor mental health days raw value',
 'Poor mental health days numerator',
 'Poor mental health days denominator',
 'Poor mental health days CI low',
 'Poor mental health days CI high',
 'Low birthweight raw value',
 'Low birthweight numerator',
 'Low birthw

In [317]:
df1.head()

,State FIPS Code,County FIPS Code,5-digit FIPS Code,State Abbreviation,Name,Release Year,County Ranked (Yes=1/No=0),Premature death raw value,Premature death numerator,Premature death denominator,...,Male population 18-44 raw value,Male population 45-64 raw value,Male population 65+ raw value,Total male population raw value,Female population 0-17 raw value,Female population 18-44 raw value,Female population 45-64 raw value,Female population 65+ raw value,Total female population raw value,Population growth raw value
0,1,0,1000,AL,Alabama,2019,NaN,9917.232898,80440.0,13636816.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,1001,AL,Autauga County,2019,1.0,8824.057123,815.0,156132.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,3,1003,AL,Baldwin County,2019,1.0,7224.632160,2827.0,576496.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,5,1005,AL,Barbour County,2019,1.0,9586.165037,451.0,72222.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,7,1007,AL,Bibb County,2019,1.0,11783.543675,445.0,63653.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [318]:
df1.iloc[:,7:12].head()

,Premature death raw value,Premature death numerator,Premature death denominator,Premature death CI low,Premature death CI high
0,9917.232898,80440.0,13636816.0,9815.190945,10019.274852
1,8824.057123,815.0,156132.0,7935.325830,9712.788417
2,7224.632160,2827.0,576496.0,6794.128380,7655.135940
3,9586.165037,451.0,72222.0,8200.117541,10972.212533
4,11783.543675,445.0,63653.0,10159.978321,13407.109028


In [336]:
# Remove unnecessary columns

filter_out = ['numerator', 'denominator', 'CI low', 'CI high']
df2 = df1.copy()
for f in filter_out:
    df2 = df2.loc[:, ~df2.columns.str.contains(f)]

# Clean up names

replace_dict = {' raw value':'', ' - ':'_', '-':'_', '=':'', '/':'_',
                '(':'', ')':'', '.':'', '+':' above', '%':'Percent', ' ':'_'}

for key, value in replace_dict.items():
    df2.columns = df2.columns.str.replace(key, value)

# Drop column if data is > 50% null

df2 = df2.drop(df2.loc[:,list((100*(df2.isnull().sum()/len(df2.index))>50))].columns, 1)

display(df2.shape)
df2.isna().sum()

(3193, 98)

State_FIPS_Code                                                            0
County_FIPS_Code                                                           0
5_digit_FIPS_Code                                                          0
State_Abbreviation                                                         0
Name                                                                       0
Release_Year                                                               0
County_Ranked_Yes1_No0                                                    51
Premature_death                                                           61
Poor_or_fair_health                                                        0
Poor_physical_health_days                                                  0
Poor_mental_health_days                                                    0
Low_birthweight                                                          107
Low_birthweight_White                                                   1376

### Excel file

In [30]:
xl = pd.ExcelFile('2019 County Health Rankings Data - v2.xls')

In [31]:
sheets = xl.sheet_names
sheets

['Introduction',
 'Outcomes & Factors Rankings',
 'Outcomes & Factors SubRankings',
 'Ranked Measure Data',
 'Ranked Measure Sources & Years',
 'Additional Measure Data',
 'Addtl Measure Sources & Years']

In [297]:
xl1 = xl.parse(sheets[1], skiprows=1)
xl1.head()

,FIPS,State,County,# of Ranked Counties,Rank,Quartile,Rank.1,Quartile.1
0,1001,Alabama,Autauga,67,7,1,11,1
1,1003,Alabama,Baldwin,67,2,1,3,1
2,1005,Alabama,Barbour,67,33,2,58,4
3,1007,Alabama,Bibb,67,40,3,29,2
4,1009,Alabama,Blount,67,22,2,16,1


In [298]:
xl1.shape

(3142, 8)

In [299]:
xl3 = xl.parse(sheets[3], header=[0,1])
xl3.head()

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                FIPS              State             County   
0               1001            Alabama            Autauga   
1               1003            Alabama            Baldwin   
2               1005            Alabama            Barbour   
3               1007            Alabama               Bibb   
4               1009            Alabama             Blount   

                    Premature death                                       \
  Years of Potential Life Lost Rate  95% CI - Low 95% CI - High Quartile   
0                       8824.057123   7935.325829   9712.788417        1   
1                       7224.632160   6794.128380   7655.135940        1   
2                       9586.165037   8200.117541  10972.212533        1   
3                      11783.543675  10159.978321  13407.109028        3   
4                      10908.101822   9895.582572  11920.621071        3   

                                                            ...  \
  YPLL Rate (Black) YPLL Rate (Hispanic) YPLL Rate (White)  ...   
0      10471.252986                  NaN       8706.658832  ...   
1      10042.472874          3086.605695       7277.780727  ...   
2      11332.562909                  NaN       7309.636719  ...   
3      14812.539280                  NaN      11327.563749  ...   
4               NaN          5619.645186      11336.046321  ...   

  Driving alone to work                                 \
          95% CI - High Quartile % Drive Alone (Black)   
0             88.900847        3             82.722037   
1             86.570681        2             84.921904   
2             85.769343        1             80.154701   
3             90.677264        3                   NaN   
4             88.846801        3                   NaN   

                                                 Long commute - driving alone  \
  % Drive Alone (Hispanic) % Drive Alone (White)    # Workers who Drive Alone   
0                      NaN             83.710021                        20911   
1                68.343876             83.799135                        74415   
2                      NaN             86.245265                         7242   
3                      NaN                   NaN                         6930   
4                81.767181             83.414506                        18426   

                                                                     
  % Long Commute - Drives Alone 95% CI - Low 95% CI - High Quartile  
0                          38.3    34.362673     42.237327        3  
1                          40.5    38.183480     42.816520        3  
2                          33.8    28.867176     38.732824        2  
3                          48.6    40.310796     56.889204        4  
4                          59.7    55.480714     63.919286        4  

[5 rows x 162 columns]

In [300]:
xl3['Premature death'].Quartile.head()

0    1
1    1
2    1
3    3
4    3
Name: Quartile, dtype: object